# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [1510]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [1511]:
portfolio.head(10)

reward                      channels  difficulty  duration     offer_type  \
0      10       [email, mobile, social]          10         7           bogo   
1      10  [web, email, mobile, social]          10         5           bogo   
2       0          [web, email, mobile]           0         4  informational   
3       5          [web, email, mobile]           5         7           bogo   
4       5                  [web, email]          20        10       discount   
5       3  [web, email, mobile, social]           7         7       discount   
6       2  [web, email, mobile, social]          10        10       discount   
7       0       [email, mobile, social]           0         3  informational   
8       5  [web, email, mobile, social]           5         5           bogo   
9       2          [web, email, mobile]          10         7       discount   

                                 id  
0  ae264e3637204a6fb9bb56bc8210ddfd  
1  4d5c57ea9a6940dd891ad53e9dbe8da0  
2  3f207df678b143eea3cee63160fa8bed  
3  9b98b8c7a33c4b65b9aebfe6a799e6d9  
4  0b1e1539f2cc45b7b9fa7c272da2e1d7  
5  2298d6c36e964ae4a3e7e9706d1fb8c2  
6  fafdcd668e3743c1bb461111dcafc2a4  
7  5a8bc65990b245e5a138643cd4eb9837  
8  f19421c1d4aa40978ebb69ca19b0e20d  
9  2906b810c7d4411798c6938adc9daaa5

In [1512]:
profile.head()

gender  age                                id  became_member_on    income
0   None  118  68be06ca386d4c31939f3a4f0e3dd783          20170212       NaN
1      F   55  0610b486422d4921ae7d2bf64640c50b          20170715  112000.0
2   None  118  38fe809add3b4fcf9315a9694bb96ff5          20180712       NaN
3      F   75  78afa995795e4d85b5d9ceeca43f5fef          20170509  100000.0
4   None  118  a03223e636434f42ac4c3df47e8bac43          20170804       NaN

In [1513]:
transcript.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0  
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0  
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0

In [1514]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

#### Transcript file's event columns has four categories: 
* transaction
* offer received
* offer viewed
* offer completed

lets group by according to event type and extract the corresponding values

In [1515]:
## Creating separate dataframes for each event type for easy sorting of values
event=transcript.groupby('event')
transaction_df=event.get_group('transaction')
offer_received_df=event.get_group('offer received')
offer_viewed_df=event.get_group('offer viewed')
offer_completed_df=event.get_group('offer completed')

In [1516]:
transaction_df.head()

person        event  \
12654  02c083884c7d45b39cc68e1314fec56c  transaction   
12657  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction   
12659  54890f68699049c2a04d415abc25e717  transaction   
12670  b2f1cd155b864803ad8334cdf13c4bd2  transaction   
12671  fe97aa22dd3e48c8b143116a8403dd52  transaction   

                                value  time  
12654  {'amount': 0.8300000000000001}     0  
12657               {'amount': 34.56}     0  
12659               {'amount': 13.23}     0  
12670               {'amount': 19.51}     0  
12671               {'amount': 18.97}     0

In [1517]:
offer_completed_df.head()

person            event  \
12658  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed   
12672  fe97aa22dd3e48c8b143116a8403dd52  offer completed   
12679  629fc02d56414d91bca360decdfa9288  offer completed   
12692  676506bad68e4161b9bbaffeb039626b  offer completed   
12697  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed   

                                                   value  time  
12658  {'offer_id': '2906b810c7d4411798c6938adc9daaa5...     0  
12672  {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...     0  
12679  {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...     0  
12692  {'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...     0  
12697  {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...     0

In [1518]:
offer_viewed_df.head()

person         event  \
12650  389bc3fa690240e798340f5a15918d5c  offer viewed   
12651  d1ede868e29245ea91818a903fec04c6  offer viewed   
12652  102e9454054946fda62242d2e176fdce  offer viewed   
12653  02c083884c7d45b39cc68e1314fec56c  offer viewed   
12655  be8a5d1981a2458d90b255ddc7e0d174  offer viewed   

                                                  value  time  
12650  {'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}     0  
12651  {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}     0  
12652  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0  
12653  {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}     0  
12655  {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}     0

In [1519]:
offer_received_df.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0  
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0  
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0

## Values
* Offer Viewed : offer_id
* Offer Received: offer_id
* Offer Completed: offer_id
* transaction: amount

### On using str.replace on 'value' column, it was returning Nan values, therefore google helped me solve this problem [https://stackoverflow.com/questions/50529269/pandas-str-split-returning-nan]

In [1520]:
amt=pd.DataFrame(transaction_df['value'].values.tolist(), index=transaction_df.index)
transaction_df=pd.concat([transaction_df, amt], axis=1).drop(columns='value')

In [1521]:
offer=pd.DataFrame(offer_viewed_df['value'].values.tolist(), index=offer_viewed_df.index)
offer_viewed_df=pd.concat([offer_viewed_df, offer], axis=1).drop(columns='value')

In [1522]:
offer=pd.DataFrame(offer_received_df['value'].values.tolist(), index=offer_received_df.index)
offer_received_df=pd.concat([offer_received_df, offer], axis=1).drop(columns='value')

In [1523]:
offer=pd.DataFrame(offer_completed_df['value'].values.tolist(), index=offer_completed_df.index)
offer_completed_df=pd.concat([offer_completed_df, offer], axis=1).drop(columns='value')

In [1524]:
transaction_df.rename(columns={'time':'time_completed'}, inplace=True)
transaction_df.head()

person        event  time_completed  amount
12654  02c083884c7d45b39cc68e1314fec56c  transaction               0    0.83
12657  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction               0   34.56
12659  54890f68699049c2a04d415abc25e717  transaction               0   13.23
12670  b2f1cd155b864803ad8334cdf13c4bd2  transaction               0   19.51
12671  fe97aa22dd3e48c8b143116a8403dd52  transaction               0   18.97

In [1525]:
offer_received_df.rename(columns={'offer id':'offer_id'}, inplace=True)
offer_received_df.head()

person           event  time  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received     0   
1  a03223e636434f42ac4c3df47e8bac43  offer received     0   
2  e2127556f4f64592b11af22de27a7932  offer received     0   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received     0   
4  68617ca6246f4fbc85e91a2a49552598  offer received     0   

                           offer_id  
0  9b98b8c7a33c4b65b9aebfe6a799e6d9  
1  0b1e1539f2cc45b7b9fa7c272da2e1d7  
2  2906b810c7d4411798c6938adc9daaa5  
3  fafdcd668e3743c1bb461111dcafc2a4  
4  4d5c57ea9a6940dd891ad53e9dbe8da0

In [1526]:
offer_viewed_df.rename(columns={'offer id':'offer_id'}, inplace=True)
offer_viewed_df.head()

person         event  time  \
12650  389bc3fa690240e798340f5a15918d5c  offer viewed     0   
12651  d1ede868e29245ea91818a903fec04c6  offer viewed     0   
12652  102e9454054946fda62242d2e176fdce  offer viewed     0   
12653  02c083884c7d45b39cc68e1314fec56c  offer viewed     0   
12655  be8a5d1981a2458d90b255ddc7e0d174  offer viewed     0   

                               offer_id  
12650  f19421c1d4aa40978ebb69ca19b0e20d  
12651  5a8bc65990b245e5a138643cd4eb9837  
12652  4d5c57ea9a6940dd891ad53e9dbe8da0  
12653  ae264e3637204a6fb9bb56bc8210ddfd  
12655  5a8bc65990b245e5a138643cd4eb9837

In [1527]:
offer_completed_df.head()

person            event  time  \
12658  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed     0   
12672  fe97aa22dd3e48c8b143116a8403dd52  offer completed     0   
12679  629fc02d56414d91bca360decdfa9288  offer completed     0   
12692  676506bad68e4161b9bbaffeb039626b  offer completed     0   
12697  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed     0   

                               offer_id  reward  
12658  2906b810c7d4411798c6938adc9daaa5       2  
12672  fafdcd668e3743c1bb461111dcafc2a4       2  
12679  9b98b8c7a33c4b65b9aebfe6a799e6d9       5  
12692  ae264e3637204a6fb9bb56bc8210ddfd      10  
12697  4d5c57ea9a6940dd891ad53e9dbe8da0      10

#### Now we have dataframes of different event types and their corresponding offer_id, reward and amount

In [1528]:
portfolio

reward                      channels  difficulty  duration     offer_type  \
0      10       [email, mobile, social]          10         7           bogo   
1      10  [web, email, mobile, social]          10         5           bogo   
2       0          [web, email, mobile]           0         4  informational   
3       5          [web, email, mobile]           5         7           bogo   
4       5                  [web, email]          20        10       discount   
5       3  [web, email, mobile, social]           7         7       discount   
6       2  [web, email, mobile, social]          10        10       discount   
7       0       [email, mobile, social]           0         3  informational   
8       5  [web, email, mobile, social]           5         5           bogo   
9       2          [web, email, mobile]          10         7       discount   

                                 id  
0  ae264e3637204a6fb9bb56bc8210ddfd  
1  4d5c57ea9a6940dd891ad53e9dbe8da0  
2  3f207df678b143eea3cee63160fa8bed  
3  9b98b8c7a33c4b65b9aebfe6a799e6d9  
4  0b1e1539f2cc45b7b9fa7c272da2e1d7  
5  2298d6c36e964ae4a3e7e9706d1fb8c2  
6  fafdcd668e3743c1bb461111dcafc2a4  
7  5a8bc65990b245e5a138643cd4eb9837  
8  f19421c1d4aa40978ebb69ca19b0e20d  
9  2906b810c7d4411798c6938adc9daaa5

### Converting Channel Column in Portfolio into dummy varaibles

In [1529]:
portfolio['channel_web'] = portfolio['channels'].apply(lambda x: 1 if 'web' in x else 0)
portfolio['channel_email'] = portfolio['channels'].apply(lambda x: 1 if 'email' in x else 0)
portfolio['channel_social'] = portfolio['channels'].apply(lambda x: 1 if 'social' in x else 0)
portfolio['channel_mobile'] = portfolio['channels'].apply(lambda x: 1 if 'mobile' in x else 0)

In [1530]:
portfolio.drop(columns='channels', inplace=True)

### Preparing for Merging of  Offer received, viewed and completed DataFrames to Portfolio DF

In [1531]:
portfolio.rename(columns={'id':'offer_id'}, inplace=True)

In [1532]:
#converting duration into hours
portfolio['duration']=portfolio['duration']*24

In [1533]:
portfolio['least_pay']=np.where(portfolio['offer_type']=='discount', (portfolio['difficulty']-portfolio['reward']),portfolio['difficulty'])

In [1534]:
portfolio.head(10)

reward  difficulty  duration     offer_type  \
0      10          10       168           bogo   
1      10          10       120           bogo   
2       0           0        96  informational   
3       5           5       168           bogo   
4       5          20       240       discount   
5       3           7       168       discount   
6       2          10       240       discount   
7       0           0        72  informational   
8       5           5       120           bogo   
9       2          10       168       discount   

                           offer_id  channel_web  channel_email  \
0  ae264e3637204a6fb9bb56bc8210ddfd            0              1   
1  4d5c57ea9a6940dd891ad53e9dbe8da0            1              1   
2  3f207df678b143eea3cee63160fa8bed            1              1   
3  9b98b8c7a33c4b65b9aebfe6a799e6d9            1              1   
4  0b1e1539f2cc45b7b9fa7c272da2e1d7            1              1   
5  2298d6c36e964ae4a3e7e9706d1fb8c2            1              1   
6  fafdcd668e3743c1bb461111dcafc2a4            1              1   
7  5a8bc65990b245e5a138643cd4eb9837            0              1   
8  f19421c1d4aa40978ebb69ca19b0e20d            1              1   
9  2906b810c7d4411798c6938adc9daaa5            1              1   

   channel_social  channel_mobile  least_pay  
0               1               1         10  
1               1               1         10  
2               0               1          0  
3               0               1          5  
4               0               0         15  
5               1               1          4  
6               1               1          8  
7               1               1          0  
8               1               1          5  
9               0               1          8

In [1535]:
offer_completed_df.head()

person            event  time  \
12658  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed     0   
12672  fe97aa22dd3e48c8b143116a8403dd52  offer completed     0   
12679  629fc02d56414d91bca360decdfa9288  offer completed     0   
12692  676506bad68e4161b9bbaffeb039626b  offer completed     0   
12697  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed     0   

                               offer_id  reward  
12658  2906b810c7d4411798c6938adc9daaa5       2  
12672  fafdcd668e3743c1bb461111dcafc2a4       2  
12679  9b98b8c7a33c4b65b9aebfe6a799e6d9       5  
12692  ae264e3637204a6fb9bb56bc8210ddfd      10  
12697  4d5c57ea9a6940dd891ad53e9dbe8da0      10

In [1536]:
offer_completed_df=pd.merge(offer_completed_df,portfolio, on='offer_id', how='left').drop(columns='reward_y').rename(columns={'reward_x':'reward','event':'offer_completed'})

In [1537]:
offer_received_df=pd.merge(offer_received_df , portfolio, on='offer_id', how='left').rename(columns={'event':'offer_received'})

In [1538]:
offer_viewed_df=pd.merge(offer_viewed_df , portfolio, on='offer_id', how='left').rename(columns={'event':'offer_view'})

In [1539]:
offer_received_df.head()

person  offer_received  time  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received     0   
1  a03223e636434f42ac4c3df47e8bac43  offer received     0   
2  e2127556f4f64592b11af22de27a7932  offer received     0   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received     0   
4  68617ca6246f4fbc85e91a2a49552598  offer received     0   

                           offer_id  reward  difficulty  duration offer_type  \
0  9b98b8c7a33c4b65b9aebfe6a799e6d9       5           5       168       bogo   
1  0b1e1539f2cc45b7b9fa7c272da2e1d7       5          20       240   discount   
2  2906b810c7d4411798c6938adc9daaa5       2          10       168   discount   
3  fafdcd668e3743c1bb461111dcafc2a4       2          10       240   discount   
4  4d5c57ea9a6940dd891ad53e9dbe8da0      10          10       120       bogo   

   channel_web  channel_email  channel_social  channel_mobile  least_pay  
0            1              1               0               1          5  
1            1              1               0               0         15  
2            1              1               0               1          8  
3            1              1               1               1          8  
4            1              1               1               1         10

### Creating Dummy Variables for offer view, recieved and completed columns for easy filtering at later stages

In [1540]:
offer_completed_df['offer_completed']=offer_completed_df['offer_completed'].apply(lambda x: 4 if 'offer completed' in x else 0)

In [1541]:
offer_viewed_df['offer_view']=offer_viewed_df['offer_view'].apply(lambda x: 2 if 'offer viewed' in x else 0)

In [1542]:
offer_received_df['offer_received']=offer_received_df['offer_received'].apply(lambda x: 1 if 'offer received' in x else 0)

In [1543]:
offer_received_df.head()

person  offer_received  time  \
0  78afa995795e4d85b5d9ceeca43f5fef               1     0   
1  a03223e636434f42ac4c3df47e8bac43               1     0   
2  e2127556f4f64592b11af22de27a7932               1     0   
3  8ec6ce2a7e7949b1bf142def7d0e0586               1     0   
4  68617ca6246f4fbc85e91a2a49552598               1     0   

                           offer_id  reward  difficulty  duration offer_type  \
0  9b98b8c7a33c4b65b9aebfe6a799e6d9       5           5       168       bogo   
1  0b1e1539f2cc45b7b9fa7c272da2e1d7       5          20       240   discount   
2  2906b810c7d4411798c6938adc9daaa5       2          10       168   discount   
3  fafdcd668e3743c1bb461111dcafc2a4       2          10       240   discount   
4  4d5c57ea9a6940dd891ad53e9dbe8da0      10          10       120       bogo   

   channel_web  channel_email  channel_social  channel_mobile  least_pay  
0            1              1               0               1          5  
1            1              1               0               0         15  
2            1              1               0               1          8  
3            1              1               1               1          8  
4            1              1               1               1         10

In [1544]:
print(offer_received_df.shape, offer_viewed_df.shape, offer_completed_df.shape)

(76277, 13) (57725, 13) (33579, 13)


### Checking if one user is always being send with unique offer or not

In [1545]:
uni=offer_received_df.groupby('person')['offer_id']

In [1546]:
uni.value_counts().head(50)

person                            offer_id                        
0009655768c64bdeb2e877511632db8f  2906b810c7d4411798c6938adc9daaa5    1
                                  3f207df678b143eea3cee63160fa8bed    1
                                  5a8bc65990b245e5a138643cd4eb9837    1
                                  f19421c1d4aa40978ebb69ca19b0e20d    1
                                  fafdcd668e3743c1bb461111dcafc2a4    1
00116118485d4dfda04fdbaba9a87b5c  f19421c1d4aa40978ebb69ca19b0e20d    2
0011e0d4e6b944f998e987f904e8c1e5  0b1e1539f2cc45b7b9fa7c272da2e1d7    1
                                  2298d6c36e964ae4a3e7e9706d1fb8c2    1
                                  3f207df678b143eea3cee63160fa8bed    1
                                  5a8bc65990b245e5a138643cd4eb9837    1
                                  9b98b8c7a33c4b65b9aebfe6a799e6d9    1
0020c2b971eb4e9188eac86d93036a77  fafdcd668e3743c1bb461111dcafc2a4    2
                                  4d5c57ea9a6940dd891ad53e9dbe8da0   

### One offer can be sent to one user more than once!

In [1547]:
offer_received_df.sort_values(['person','offer_id'])

person  offer_received  time  \
66269  0009655768c64bdeb2e877511632db8f               1   576   
28096  0009655768c64bdeb2e877511632db8f               1   336   
15448  0009655768c64bdeb2e877511632db8f               1   168   
40835  0009655768c64bdeb2e877511632db8f               1   408   
53578  0009655768c64bdeb2e877511632db8f               1   504   
...                                 ...             ...   ...   
26705  ffff82501cea40309d5fdd7edcca4a07               1   336   
39464  ffff82501cea40309d5fdd7edcca4a07               1   408   
64885  ffff82501cea40309d5fdd7edcca4a07               1   576   
52200  ffff82501cea40309d5fdd7edcca4a07               1   504   
1393   ffff82501cea40309d5fdd7edcca4a07               1     0   

                               offer_id  reward  difficulty  duration  \
66269  2906b810c7d4411798c6938adc9daaa5       2          10       168   
28096  3f207df678b143eea3cee63160fa8bed       0           0        96   
15448  5a8bc65990b245e5a138643cd4eb9837       0           0        72   
40835  f19421c1d4aa40978ebb69ca19b0e20d       5           5       120   
53578  fafdcd668e3743c1bb461111dcafc2a4       2          10       240   
...                                 ...     ...         ...       ...   
26705  2906b810c7d4411798c6938adc9daaa5       2          10       168   
39464  2906b810c7d4411798c6938adc9daaa5       2          10       168   
64885  2906b810c7d4411798c6938adc9daaa5       2          10       168   
52200  9b98b8c7a33c4b65b9aebfe6a799e6d9       5           5       168   
1393   fafdcd668e3743c1bb461111dcafc2a4       2          10       240   

          offer_type  channel_web  channel_email  channel_social  \
66269       discount            1              1               0   
28096  informational            1              1               0   
15448  informational            0              1               1   
40835           bogo            1              1               1   
53578       discount            1              1               1   
...              ...          ...            ...             ...   
26705       discount            1              1               0   
39464       discount            1              1               0   
64885       discount            1              1               0   
52200           bogo            1              1               0   
1393        discount            1              1               1   

       channel_mobile  least_pay  
66269               1          8  
28096               1          0  
15448               1          0  
40835               1          5  
53578               1          8  
...               ...        ...  
26705               1          8  
39464               1          8  
64885               1          8  
52200               1          5  
1393                1          8  

[76277 rows x 13 columns]

In [1548]:
offer_viewed_df.sort_values(['person','offer_id']).head(10)

person  offer_view  time  \
27092  0009655768c64bdeb2e877511632db8f           2   372   
15841  0009655768c64bdeb2e877511632db8f           2   192   
36935  0009655768c64bdeb2e877511632db8f           2   456   
45628  0009655768c64bdeb2e877511632db8f           2   540   
17854  00116118485d4dfda04fdbaba9a87b5c           2   216   
55801  00116118485d4dfda04fdbaba9a87b5c           2   630   
34923  0011e0d4e6b944f998e987f904e8c1e5           2   432   
14985  0011e0d4e6b944f998e987f904e8c1e5           2   186   
2426   0011e0d4e6b944f998e987f904e8c1e5           2     6   
25013  0011e0d4e6b944f998e987f904e8c1e5           2   354   

                               offer_id  reward  difficulty  duration  \
27092  3f207df678b143eea3cee63160fa8bed       0           0        96   
15841  5a8bc65990b245e5a138643cd4eb9837       0           0        72   
36935  f19421c1d4aa40978ebb69ca19b0e20d       5           5       120   
45628  fafdcd668e3743c1bb461111dcafc2a4       2          10       240   
17854  f19421c1d4aa40978ebb69ca19b0e20d       5           5       120   
55801  f19421c1d4aa40978ebb69ca19b0e20d       5           5       120   
34923  0b1e1539f2cc45b7b9fa7c272da2e1d7       5          20       240   
14985  2298d6c36e964ae4a3e7e9706d1fb8c2       3           7       168   
2426   3f207df678b143eea3cee63160fa8bed       0           0        96   
25013  5a8bc65990b245e5a138643cd4eb9837       0           0        72   

          offer_type  channel_web  channel_email  channel_social  \
27092  informational            1              1               0   
15841  informational            0              1               1   
36935           bogo            1              1               1   
45628       discount            1              1               1   
17854           bogo            1              1               1   
55801           bogo            1              1               1   
34923       discount            1              1               0   
14985       discount            1              1               1   
2426   informational            1              1               0   
25013  informational            0              1               1   

       channel_mobile  least_pay  
27092               1          0  
15841               1          0  
36935               1          5  
45628               1          8  
17854               1          5  
55801               1          5  
34923               0         15  
14985               1          4  
2426                1          0  
25013               1          0

In [1549]:
transaction_df.head()

person        event  time_completed  amount
12654  02c083884c7d45b39cc68e1314fec56c  transaction               0    0.83
12657  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction               0   34.56
12659  54890f68699049c2a04d415abc25e717  transaction               0   13.23
12670  b2f1cd155b864803ad8334cdf13c4bd2  transaction               0   19.51
12671  fe97aa22dd3e48c8b143116a8403dd52  transaction               0   18.97

In [1550]:
offer_completed_df.sort_values(['person','offer_id'])

person  offer_completed  time  \
25942  0009655768c64bdeb2e877511632db8f                4   576   
14758  0009655768c64bdeb2e877511632db8f                4   414   
22811  0009655768c64bdeb2e877511632db8f                4   528   
25955  0011e0d4e6b944f998e987f904e8c1e5                4   576   
8737   0011e0d4e6b944f998e987f904e8c1e5                4   252   
...                                 ...              ...   ...   
13021  ffff82501cea40309d5fdd7edcca4a07                4   384   
14693  ffff82501cea40309d5fdd7edcca4a07                4   414   
25850  ffff82501cea40309d5fdd7edcca4a07                4   576   
20522  ffff82501cea40309d5fdd7edcca4a07                4   504   
2518   ffff82501cea40309d5fdd7edcca4a07                4    60   

                               offer_id  reward  difficulty  duration  \
25942  2906b810c7d4411798c6938adc9daaa5       2          10       168   
14758  f19421c1d4aa40978ebb69ca19b0e20d       5           5       120   
22811  fafdcd668e3743c1bb461111dcafc2a4       2          10       240   
25955  0b1e1539f2cc45b7b9fa7c272da2e1d7       5          20       240   
8737   2298d6c36e964ae4a3e7e9706d1fb8c2       3           7       168   
...                                 ...     ...         ...       ...   
13021  2906b810c7d4411798c6938adc9daaa5       2          10       168   
14693  2906b810c7d4411798c6938adc9daaa5       2          10       168   
25850  2906b810c7d4411798c6938adc9daaa5       2          10       168   
20522  9b98b8c7a33c4b65b9aebfe6a799e6d9       5           5       168   
2518   fafdcd668e3743c1bb461111dcafc2a4       2          10       240   

      offer_type  channel_web  channel_email  channel_social  channel_mobile  \
25942   discount            1              1               0               1   
14758       bogo            1              1               1               1   
22811   discount            1              1               1               1   
25955   discount            1              1               0               0   
8737    discount            1              1               1               1   
...          ...          ...            ...             ...             ...   
13021   discount            1              1               0               1   
14693   discount            1              1               0               1   
25850   discount            1              1               0               1   
20522       bogo            1              1               0               1   
2518    discount            1              1               1               1   

       least_pay  
25942          8  
14758          5  
22811          8  
25955         15  
8737           4  
...          ...  
13021          8  
14693          8  
25850          8  
20522          5  
2518           8  

[33579 rows x 13 columns]

In [1551]:
offer_received_df.isnull().sum()

person            0
offer_received    0
time              0
offer_id          0
reward            0
difficulty        0
duration          0
offer_type        0
channel_web       0
channel_email     0
channel_social    0
channel_mobile    0
least_pay         0
dtype: int64

### Combining all three dataframe (received, viewed and completed ) into one

In [1552]:
offers=pd.merge(offer_received_df, offer_viewed_df, on=['person','offer_id','reward', 'difficulty',
                                                 'duration', 'offer_type', 'channel_web', 'channel_email',
                                                 'channel_social', 'channel_mobile'], 
         how='left').drop_duplicates(subset=['person','offer_id','time_x'])
offers=pd.merge(offers, offer_completed_df, on=['person','offer_id','reward', 'difficulty',
       'duration', 'offer_type', 'channel_web', 'channel_email',
       'channel_social', 'channel_mobile'], how='left').drop_duplicates(subset=['person','offer_id','time_x'])
offers.isnull().sum()

person                 0
offer_received         0
time_x                 0
offer_id               0
reward                 0
difficulty             0
duration               0
offer_type             0
channel_web            0
channel_email          0
channel_social         0
channel_mobile         0
least_pay_x            0
offer_view         15992
time_y             15992
least_pay_y        15992
offer_completed    40280
time               40280
least_pay          40280
dtype: int64

In [1553]:
offers.shape

(76277, 19)

In [1554]:
offers.rename(columns={'time_x':'time_start','time_y':'time_view','time':'time_completed'}, inplace=True)

In [1555]:
offers=offers.reindex(sorted(offers.columns), axis=1)

### Separating Data frame for offers that went waste
* Offer was received but not viewed and completed
* Offer was received NOT Viewed but Completed (User Purchase was not due to Offer)
* Offer was received, viewed but Not Completed (Excluding offers of offer_type: informational)

In [1556]:
offers['offer_view']=offers['offer_view'].fillna(0.0)
offers['offer_completed']=offers['offer_completed'].fillna(0.0)

In [1557]:
offers.sort_values(['person','time_completed']).head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
46246              1               1               1            1           5   
60689              1               1               1            1          10   
75133              1               1               0            1          10   
17397              1               1               1            0           0   
31703              1               1               0            1           0   

       duration  least_pay  least_pay_x  least_pay_y  offer_completed  \
46246       120        5.0            5          5.0              4.0   
60689       240        8.0            8          8.0              4.0   
75133       168        8.0            8          NaN              4.0   
17397        72        NaN            0          0.0              0.0   
31703        96        NaN            0          0.0              0.0   

                               offer_id  offer_received     offer_type  \
46246  f19421c1d4aa40978ebb69ca19b0e20d               1           bogo   
60689  fafdcd668e3743c1bb461111dcafc2a4               1       discount   
75133  2906b810c7d4411798c6938adc9daaa5               1       discount   
17397  5a8bc65990b245e5a138643cd4eb9837               1  informational   
31703  3f207df678b143eea3cee63160fa8bed               1  informational   

       offer_view                            person  reward  time_completed  \
46246         2.0  0009655768c64bdeb2e877511632db8f       5           414.0   
60689         2.0  0009655768c64bdeb2e877511632db8f       2           528.0   
75133         0.0  0009655768c64bdeb2e877511632db8f       2           576.0   
17397         2.0  0009655768c64bdeb2e877511632db8f       0             NaN   
31703         2.0  0009655768c64bdeb2e877511632db8f       0             NaN   

       time_start  time_view  
46246         408      456.0  
60689         504      540.0  
75133         576        NaN  
17397         168      192.0  
31703         336      372.0

In [1558]:
offers['offer_received'].astype('float')
offers['total']= offers['offer_received']+offers['offer_completed']+offers['offer_view']

### Adding the start time of offer and duration will help us identifying successful informational offers

In [1559]:
offers['time_validity']= (offers['time_start']+offers['duration']).astype('float')
offers.head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
0              1               1               0            1           5   
1              1               0               0            1          20   
2              1               1               0            1          10   
3              1               1               1            1          10   
4              1               1               1            1          10   

   duration  least_pay  least_pay_x  least_pay_y  offer_completed  ...  \
0       168        5.0            5          5.0              4.0  ...   
1       240        NaN           15         15.0              0.0  ...   
2       168        NaN            8          8.0              0.0  ...   
3       240        NaN            8          8.0              0.0  ...   
4       120        NaN           10         10.0              0.0  ...   

  offer_received  offer_type offer_view                            person  \
0              1        bogo        2.0  78afa995795e4d85b5d9ceeca43f5fef   
1              1    discount        2.0  a03223e636434f42ac4c3df47e8bac43   
2              1    discount        2.0  e2127556f4f64592b11af22de27a7932   
3              1    discount        2.0  8ec6ce2a7e7949b1bf142def7d0e0586   
4              1        bogo        2.0  68617ca6246f4fbc85e91a2a49552598   

  reward  time_completed  time_start  time_view  total  time_validity  
0      5           132.0           0        6.0    7.0          168.0  
1      5             NaN           0        6.0    3.0          240.0  
2      2             NaN           0       18.0    3.0          168.0  
3      2             NaN           0       12.0    3.0          240.0  
4     10             NaN           0       84.0    3.0          120.0  

[5 rows x 21 columns]

### Merging offers DF and transaction DF

In [1560]:
offers=pd.merge(offers,transaction_df, on=['person','time_completed'], how='left', copy=False).drop_duplicates()

In [1561]:
offers.head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
0              1               1               0            1           5   
1              1               0               0            1          20   
2              1               1               0            1          10   
3              1               1               1            1          10   
4              1               1               1            1          10   

   duration  least_pay  least_pay_x  least_pay_y  offer_completed  ...  \
0       168        5.0            5          5.0              4.0  ...   
1       240        NaN           15         15.0              0.0  ...   
2       168        NaN            8          8.0              0.0  ...   
3       240        NaN            8          8.0              0.0  ...   
4       120        NaN           10         10.0              0.0  ...   

  offer_view                            person reward  time_completed  \
0        2.0  78afa995795e4d85b5d9ceeca43f5fef      5           132.0   
1        2.0  a03223e636434f42ac4c3df47e8bac43      5             NaN   
2        2.0  e2127556f4f64592b11af22de27a7932      2             NaN   
3        2.0  8ec6ce2a7e7949b1bf142def7d0e0586      2             NaN   
4        2.0  68617ca6246f4fbc85e91a2a49552598     10             NaN   

  time_start  time_view  total  time_validity        event  amount  
0          0        6.0    7.0          168.0  transaction   19.89  
1          0        6.0    3.0          240.0          NaN     NaN  
2          0       18.0    3.0          168.0          NaN     NaN  
3          0       12.0    3.0          240.0          NaN     NaN  
4          0       84.0    3.0          120.0          NaN     NaN  

[5 rows x 23 columns]

In [1562]:
offers.drop(columns=['event','least_pay','least_pay_y'], inplace=True)
offers=offers.rename(columns={'least_pay_x':'least_pay'})

### Some Offers are completed even after the Validity has expired, while some were viewed after the transaction was done and some were not viewed and not transacted, these are all Wasted Offers

#### Separating the waste offers and successful offers into separate DF's

In [1563]:
w1=(offers[offers['total']==1.0])
w2=(offers[(offers['total']==3.0) & (offers['offer_type']!='informational')])
w3=(offers[offers['total']==5.0])

In [1564]:
waste_offer= pd.concat([w1,w2,w3],axis=0)

In [1565]:
waste_offer=pd.concat([waste_offer,(offers[offers['time_start'] > offers['time_completed']])], axis=0)

In [1566]:
waste_offer=pd.concat([waste_offer,(offers[offers['time_completed'] > offers['time_validity']])], axis=0)
waste_offer=pd.concat([waste_offer,(offers[offers['amount'] < offers['least_pay']])], axis=0)

In [1567]:
waste_offer.drop_duplicates()
waste_offer.shape

(44357, 20)

In [1568]:
waste_offer.head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
7               1               1               0            1           0   
17              1               1               0            1          10   
25              1               1               0            1          10   
30              1               1               0            1           5   
46              1               1               1            0          10   

    duration  least_pay  offer_completed                          offer_id  \
7         96          0              0.0  3f207df678b143eea3cee63160fa8bed   
17       168          8              0.0  2906b810c7d4411798c6938adc9daaa5   
25       168          8              0.0  2906b810c7d4411798c6938adc9daaa5   
30       168          5              0.0  9b98b8c7a33c4b65b9aebfe6a799e6d9   
46       168         10              0.0  ae264e3637204a6fb9bb56bc8210ddfd   

    offer_received     offer_type  offer_view  \
7                1  informational         0.0   
17               1       discount         0.0   
25               1       discount         0.0   
30               1           bogo         0.0   
46               1           bogo         0.0   

                              person  reward  time_completed  time_start  \
7   2eeac8d8feae4a8cad5a6af0499a211d       0             NaN           0   
17  65aba5c617294649aeb624da249e1ee5       2             NaN           0   
25  25c906289d154b66bf579693f89481c9       2             NaN           0   
30  57dd18ec5ddc46828afb81ec5977bef2       5             NaN           0   
46  39dbcf43e24d41f4bbf0f134157e0e1e      10             NaN           0   

    time_view  total  time_validity  amount  
7         NaN    1.0           96.0     NaN  
17        NaN    1.0          168.0     NaN  
25        NaN    1.0          168.0     NaN  
30        NaN    1.0          168.0     NaN  
46        NaN    1.0          168.0     NaN

In [1569]:
# Creating a Dataframe in which Transactions done without viewing the offers are stored
waste_offer_transaction = waste_offer[waste_offer['amount'].notnull()]
waste_offer_transaction.drop_duplicates(inplace=True)
waste_offer.head()

/Users/aadhar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


channel_email  channel_mobile  channel_social  channel_web  difficulty  \
7               1               1               0            1           0   
17              1               1               0            1          10   
25              1               1               0            1          10   
30              1               1               0            1           5   
46              1               1               1            0          10   

    duration  least_pay  offer_completed                          offer_id  \
7         96          0              0.0  3f207df678b143eea3cee63160fa8bed   
17       168          8              0.0  2906b810c7d4411798c6938adc9daaa5   
25       168          8              0.0  2906b810c7d4411798c6938adc9daaa5   
30       168          5              0.0  9b98b8c7a33c4b65b9aebfe6a799e6d9   
46       168         10              0.0  ae264e3637204a6fb9bb56bc8210ddfd   

    offer_received     offer_type  offer_view  \
7                1  informational         0.0   
17               1       discount         0.0   
25               1       discount         0.0   
30               1           bogo         0.0   
46               1           bogo         0.0   

                              person  reward  time_completed  time_start  \
7   2eeac8d8feae4a8cad5a6af0499a211d       0             NaN           0   
17  65aba5c617294649aeb624da249e1ee5       2             NaN           0   
25  25c906289d154b66bf579693f89481c9       2             NaN           0   
30  57dd18ec5ddc46828afb81ec5977bef2       5             NaN           0   
46  39dbcf43e24d41f4bbf0f134157e0e1e      10             NaN           0   

    time_view  total  time_validity  amount  
7         NaN    1.0           96.0     NaN  
17        NaN    1.0          168.0     NaN  
25        NaN    1.0          168.0     NaN  
30        NaN    1.0          168.0     NaN  
46        NaN    1.0          168.0     NaN

In [1570]:
# Dropping all the wasted offers rows
offer_success=offers.copy()
offer_success=offer_success.drop((offer_success[(offer_success['total']==3.0) & (offer_success['offer_type']!='informational')]).index)
offer_success=offer_success.drop((offer_success[offer_success['total']==1.0]).index)
offer_success=offer_success.drop((offer_success[offer_success['total']==5.0]).index)
offer_success=offer_success.drop(offer_success[(offer_success['time_completed'])>(offer_success['time_validity'])].index)
offer_success=offer_success.drop(offer_success[(offer_success['time_start']) > (offer_success['time_completed'])].index)
offer_success=offer_success.drop(offer_success[(offer_success['amount']) < (offer_success['least_pay'])].index)

In [1571]:
offer_success.head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
0               1               1               0            1           5   
5               1               1               1            1           5   
13              1               1               0            1           0   
14              1               0               0            1          20   
15              1               1               1            1          10   

    duration  least_pay  offer_completed                          offer_id  \
0        168          5              4.0  9b98b8c7a33c4b65b9aebfe6a799e6d9   
5        120          5              4.0  f19421c1d4aa40978ebb69ca19b0e20d   
13        96          0              0.0  3f207df678b143eea3cee63160fa8bed   
14       240         15              4.0  0b1e1539f2cc45b7b9fa7c272da2e1d7   
15       240          8              4.0  fafdcd668e3743c1bb461111dcafc2a4   

    offer_received     offer_type  offer_view  \
0                1           bogo         2.0   
5                1           bogo         2.0   
13               1  informational         2.0   
14               1       discount         2.0   
15               1       discount         2.0   

                              person  reward  time_completed  time_start  \
0   78afa995795e4d85b5d9ceeca43f5fef       5           132.0           0   
5   389bc3fa690240e798340f5a15918d5c       5            60.0           0   
13  c27e0d6ab72c455a8bb66d980963de60       0             NaN           0   
14  d53717f5400c4e84affdaeda9dd926b3       5            30.0           0   
15  f806632c011441378d4646567f357a21       2           126.0           0   

    time_view  total  time_validity  amount  
0         6.0    7.0          168.0   19.89  
5         0.0    7.0          120.0    9.54  
13       18.0    3.0           96.0     NaN  
14       30.0    7.0          240.0  339.69  
15       18.0    7.0          240.0   28.29

In [1572]:
waste_offer_transaction.drop_duplicates(subset=['person','offer_id','time_completed','time_view','amount'],inplace=True)

/Users/aadhar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1573]:
waste_offer.drop_duplicates(subset=['person','offer_id','time_completed','time_view','amount'],inplace=True)

### Assumption: Since time_completed associated to Informational type offer is NaN, therefore, assuming that, for all the rows of  Informational type offer, when transaction time is in between time_view and time_validity, it would be considered successful for the particular informational type offer

In [1574]:
info=offer_success[offer_success['offer_type']=='informational']

In [1575]:
#dropping rows of Informational offer type from Offer_success Df
offer_success.drop((offer_success[offer_success['offer_type']=='informational']).index, inplace=True)

In [1576]:
# Removing transactions that are assigned to succesful offers DF and Wasted Offers Df
tran1= pd.merge(transaction_df, offer_success, on=['person','time_completed'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True).dropna(axis=1)
tran1.rename(columns={'amount_x':'amount'},inplace=True)

In [1577]:
# Removing transaction that were assigned to waste offers df, since they were done without being Viewed
tran2=pd.merge(tran1, waste_offer_transaction, 
               on=['person','time_completed'], how='outer', 
               indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
tran2= tran2.dropna(subset=['amount_x'])

In [1578]:
tran2=tran2.dropna(axis=1)
tran2.rename(columns={'amount_x':'amount'},inplace=True)

#### Difference between two DF's : Found on :https://stackoverflow.com/questions/44546086/remove-one-dataframe-from-another-with-pandas?rq=1

In [1579]:
info=pd.merge(info, tran2, on=['person'], how='right', copy=False).drop(columns=['event','amount_x'])

In [1580]:
info=info[info['time_completed_y'].ge(info['time_view']) & info['time_completed_y'].le(info['time_validity'])]

In [1581]:
info.drop(columns=['time_completed_x'], inplace=True)
info.rename(columns={'time_completed_y':'time_completed','amount_y':'amount'}, inplace=True)

In [1582]:
#Since Many Informational Offers can be sent to one user and transaction could have been done within the time frame,
#so it would be unfair to assign one amount to success of all informational offers, instead amount will be assigned 
#to just one, but others are only marked as Success but not amount
info.loc[info.duplicated(subset=['offer_id','person','time_view','time_completed','amount','reward']),['amount']]=0

In [1583]:
offer_success=pd.concat([offer_success,info], ignore_index=True).drop_duplicates()

In [1584]:
offer_success.drop_duplicates(subset=['offer_id','person','time_view','time_start'],inplace=True)
offer_success.drop_duplicates(subset=['offer_id','person','time_completed','amount'],inplace=True)

In [1585]:
offer_success.shape

(28587, 20)

In [1586]:
offer_success.sort_values(by=['person','offer_id','amount'])

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
28484            1.0             1.0             1.0          0.0         0.0   
12795            1.0             1.0             1.0          1.0         5.0   
16562            1.0             1.0             1.0          1.0        10.0   
12891            1.0             0.0             0.0          1.0        20.0   
5115             1.0             1.0             1.0          1.0         7.0   
...              ...             ...             ...          ...         ...   
68               1.0             1.0             1.0          1.0         5.0   
4552             1.0             0.0             0.0          1.0        20.0   
8672             1.0             1.0             0.0          1.0        10.0   
16160            1.0             1.0             0.0          1.0         5.0   
441              1.0             1.0             1.0          1.0        10.0   

       duration  least_pay  offer_completed                          offer_id  \
28484      72.0        0.0              0.0  5a8bc65990b245e5a138643cd4eb9837   
12795     120.0        5.0              4.0  f19421c1d4aa40978ebb69ca19b0e20d   
16562     240.0        8.0              4.0  fafdcd668e3743c1bb461111dcafc2a4   
12891     240.0       15.0              4.0  0b1e1539f2cc45b7b9fa7c272da2e1d7   
5115      168.0        4.0              4.0  2298d6c36e964ae4a3e7e9706d1fb8c2   
...         ...        ...              ...                               ...   
68        120.0        5.0              4.0  f19421c1d4aa40978ebb69ca19b0e20d   
4552      240.0       15.0              4.0  0b1e1539f2cc45b7b9fa7c272da2e1d7   
8672      168.0        8.0              4.0  2906b810c7d4411798c6938adc9daaa5   
16160     168.0        5.0              4.0  9b98b8c7a33c4b65b9aebfe6a799e6d9   
441       240.0        8.0              4.0  fafdcd668e3743c1bb461111dcafc2a4   

       offer_received     offer_type  offer_view  \
28484             1.0  informational         2.0   
12795             1.0           bogo         2.0   
16562             1.0       discount         2.0   
12891             1.0       discount         2.0   
5115              1.0       discount         2.0   
...               ...            ...         ...   
68                1.0           bogo         2.0   
4552              1.0       discount         2.0   
8672              1.0       discount         2.0   
16160             1.0           bogo         2.0   
441               1.0       discount         2.0   

                                 person  reward  time_completed  time_start  \
28484  0009655768c64bdeb2e877511632db8f     0.0           228.0       168.0   
12795  0009655768c64bdeb2e877511632db8f     5.0           414.0       408.0   
16562  0009655768c64bdeb2e877511632db8f     2.0           528.0       504.0   
12891  0011e0d4e6b944f998e987f904e8c1e5     5.0           576.0       408.0   
5115   0011e0d4e6b944f998e987f904e8c1e5     3.0           252.0       168.0   
...                                 ...     ...             ...         ...   
68     fffad4f4828548d1b5583907f2e9906b     5.0            36.0         0.0   
4552   ffff82501cea40309d5fdd7edcca4a07     5.0           198.0       168.0   
8672   ffff82501cea40309d5fdd7edcca4a07     2.0           384.0       336.0   
16160  ffff82501cea40309d5fdd7edcca4a07     5.0           504.0       504.0   
441    ffff82501cea40309d5fdd7edcca4a07     2.0            60.0         0.0   

       time_view  total  time_validity  amount  
28484      192.0    3.0          240.0   22.16  
12795      456.0    7.0          528.0    8.57  
16562      540.0    7.0          744.0   14.11  
12891      432.0    7.0          648.0   22.05  
5115       186.0    7.0          336.0   11.93  
...          ...    ...            ...     ...  
68           6.0    7.0          120.0    6.97  
4552       174.0    7.0          408.0   22.88  
8672       354.0    7.0          504.0   15.57

In [1587]:
offer_success.isnull().sum()

channel_email      0
channel_mobile     0
channel_social     0
channel_web        0
difficulty         0
duration           0
least_pay          0
offer_completed    0
offer_id           0
offer_received     0
offer_type         0
offer_view         0
person             0
reward             0
time_completed     0
time_start         0
time_view          0
total              0
time_validity      0
amount             0
dtype: int64

In [1588]:
offer_success.head()

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
0            1.0             1.0             0.0          1.0         5.0   
1            1.0             1.0             1.0          1.0         5.0   
2            1.0             0.0             0.0          1.0        20.0   
3            1.0             1.0             1.0          1.0        10.0   
4            1.0             0.0             0.0          1.0        20.0   

   duration  least_pay  offer_completed                          offer_id  \
0     168.0        5.0              4.0  9b98b8c7a33c4b65b9aebfe6a799e6d9   
1     120.0        5.0              4.0  f19421c1d4aa40978ebb69ca19b0e20d   
2     240.0       15.0              4.0  0b1e1539f2cc45b7b9fa7c272da2e1d7   
3     240.0        8.0              4.0  fafdcd668e3743c1bb461111dcafc2a4   
4     240.0       15.0              4.0  0b1e1539f2cc45b7b9fa7c272da2e1d7   

   offer_received offer_type  offer_view                            person  \
0             1.0       bogo         2.0  78afa995795e4d85b5d9ceeca43f5fef   
1             1.0       bogo         2.0  389bc3fa690240e798340f5a15918d5c   
2             1.0   discount         2.0  d53717f5400c4e84affdaeda9dd926b3   
3             1.0   discount         2.0  f806632c011441378d4646567f357a21   
4             1.0   discount         2.0  d058f73bf8674a26a95227db098147b1   

   reward  time_completed  time_start  time_view  total  time_validity  amount  
0     5.0           132.0         0.0        6.0    7.0          168.0   19.89  
1     5.0            60.0         0.0        0.0    7.0          120.0    9.54  
2     5.0            30.0         0.0       30.0    7.0          240.0  339.69  
3     2.0           126.0         0.0       18.0    7.0          240.0   28.29  
4     5.0           192.0         0.0       30.0    7.0          240.0   27.23

### Now we have all the offers that were successful and their respective amount transactions

### Creating a User based amount spend dataframe (Profile_amount)

In [1589]:
a=transaction_df.groupby('person')['amount'].sum()

In [1590]:
profile.rename(columns={'id':'person'}, inplace=True)

In [1591]:
profile_amount = pd.merge(profile,a, on='person', how='left')

### Cleaning Profile_amount DF

In [1592]:
profile_amount['age']=profile_amount['age'].replace(118,np.nan)

In [1593]:
profile_amount['gender'].fillna(np.nan, inplace=True)

In [1594]:
profile_amount['amount'].fillna(np.nan, inplace=True)

In [1595]:
#Converting joining date into date time format
profile_amount['became_member_on']=pd.to_datetime(profile_amount['became_member_on'], format='%Y%m%d')

In [1596]:
profile_amount.head()

gender   age                            person became_member_on    income  \
0    NaN   NaN  68be06ca386d4c31939f3a4f0e3dd783       2017-02-12       NaN   
1      F  55.0  0610b486422d4921ae7d2bf64640c50b       2017-07-15  112000.0   
2    NaN   NaN  38fe809add3b4fcf9315a9694bb96ff5       2018-07-12       NaN   
3      F  75.0  78afa995795e4d85b5d9ceeca43f5fef       2017-05-09  100000.0   
4    NaN   NaN  a03223e636434f42ac4c3df47e8bac43       2017-08-04       NaN   

   amount  
0   20.40  
1   77.01  
2   14.30  
3  159.27  
4    4.65

### Cleaning of profile_amt df done!

### Transactions that were made without giving offers

In [1597]:
# Finding transactions that were not made under Influence of any offer
transaction_with_offers=offer_success.append(waste_offer).drop_duplicates().reset_index()
normal_transactions=pd.merge(transaction_df, transaction_with_offers, on=['person','time_completed'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

In [1598]:
normal_transactions=normal_transactions.dropna(subset=['time_completed','amount_x']).drop_duplicates()
normal_transactions=normal_transactions.dropna(axis=1)
normal_transactions.rename(columns={'amount_x':'amount'}, inplace=True)

In [1599]:
normal_transactions.head()

person        event  time_completed  amount
0  02c083884c7d45b39cc68e1314fec56c  transaction             0.0    0.83
1  54890f68699049c2a04d415abc25e717  transaction             0.0   13.23
2  b2f1cd155b864803ad8334cdf13c4bd2  transaction             0.0   19.51
3  bbeb54e861614fc7b22a8844f72dca6c  transaction             0.0    0.22
4  a97e6f33219c432db82acfa0d19c602d  transaction             0.0   18.59

### Exporting DataFrames into CSV Format

In [1600]:
offer_success['offer_completed']=1

In [1601]:
waste_offer.loc[waste_offer['offer_completed']==4, 'offer_completed']=1

In [1602]:
waste_offer_transaction.loc[:,'offer_completed']=1

/Users/aadhar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [1603]:
waste_offer_transaction

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
20                 1               1               0            1          10   
23                 1               1               0            1          10   
62                 1               1               0            1          10   
81                 1               1               0            1           5   
101                1               0               0            1          20   
...              ...             ...             ...          ...         ...   
76050              1               1               0            1          10   
76117              1               1               1            1           7   
76199              1               0               0            1          20   
76209              1               1               1            1           7   
76269              1               0               0            1          20   

       duration  least_pay  offer_completed                          offer_id  \
20          168          8                1  2906b810c7d4411798c6938adc9daaa5   
23          168          8                1  2906b810c7d4411798c6938adc9daaa5   
62          168          8                1  2906b810c7d4411798c6938adc9daaa5   
81          168          5                1  9b98b8c7a33c4b65b9aebfe6a799e6d9   
101         240         15                1  0b1e1539f2cc45b7b9fa7c272da2e1d7   
...         ...        ...              ...                               ...   
76050       168          8                1  2906b810c7d4411798c6938adc9daaa5   
76117       168          4                1  2298d6c36e964ae4a3e7e9706d1fb8c2   
76199       240         15                1  0b1e1539f2cc45b7b9fa7c272da2e1d7   
76209       168          4                1  2298d6c36e964ae4a3e7e9706d1fb8c2   
76269       240         15                1  0b1e1539f2cc45b7b9fa7c272da2e1d7   

       offer_received offer_type  offer_view  \
20                  1   discount         0.0   
23                  1   discount         0.0   
62                  1   discount         0.0   
81                  1       bogo         0.0   
101                 1   discount         0.0   
...               ...        ...         ...   
76050               1   discount         2.0   
76117               1   discount         2.0   
76199               1   discount         2.0   
76209               1   discount         2.0   
76269               1   discount         2.0   

                                 person  reward  time_completed  time_start  \
20     868317b9be554cb18e50bc68484749a2       2           132.0           0   
23     4beeb3ed64dd4898b0edf2f6b67426d3       2           120.0           0   
62     8b70ca1f5f5a45bdac2ac2f990b71526       2           168.0           0   
81     52f0bd64403f4958ab8fe9a6947c13c3       5           582.0           0   
101    88baa20c29a94178a43a7d68e5f039d4       5           132.0           0   
...                                 ...     ...             ...         ...   
76050  cc251fdc5cc24936a741924a785ede5e       2           690.0         576   
76117  cd12501f7dfb46e6a8abe143797ae287       3           696.0         576   
76199  29b2d42d18e24a68960fdd073d95fb48       5           690.0         576   
76209  9215974995e746d79398eb22de1c9742       3           654.0         576   
76269  670626b55bfb4ba39c85b27cc7cca527       5           666.0         576   

       time_view  total  time_validity  amount  
20           NaN    5.0          168.0   12.03  
23           NaN    5.0          168.0    2.20  
62           NaN    5.0          168.0   29.44  
81           NaN    5.0          168.0    9.20  
101          NaN    5.0          240.0   25.73  
...          ...    ...            ...     ...  
76050      594.0    7.0          744.0    4.66  
76117      624.0    7.0          744.0    2.54  
76199      642.0    7.0          816.0   14.22  
76209      636.0    7.0          744.0    3.0

In [1604]:
offer_success[offer_success.duplicated(subset=['person','amount','time_completed'])]

channel_email  channel_mobile  channel_social  channel_web  difficulty  \
4086             1.0             1.0             0.0          1.0        10.0   
4093             1.0             1.0             0.0          1.0        10.0   
4138             1.0             1.0             1.0          1.0         5.0   
4198             1.0             1.0             1.0          0.0        10.0   
4230             1.0             1.0             1.0          1.0         5.0   
...              ...             ...             ...          ...         ...   
34489            1.0             1.0             1.0          0.0         0.0   
34525            1.0             1.0             1.0          0.0         0.0   
36096            1.0             1.0             0.0          1.0         0.0   
36568            1.0             1.0             1.0          0.0         0.0   
36772            1.0             1.0             1.0          0.0         0.0   

       duration  least_pay  offer_completed                          offer_id  \
4086      168.0        8.0                1  2906b810c7d4411798c6938adc9daaa5   
4093      168.0        8.0                1  2906b810c7d4411798c6938adc9daaa5   
4138      120.0        5.0                1  f19421c1d4aa40978ebb69ca19b0e20d   
4198      168.0       10.0                1  ae264e3637204a6fb9bb56bc8210ddfd   
4230      120.0        5.0                1  f19421c1d4aa40978ebb69ca19b0e20d   
...         ...        ...              ...                               ...   
34489      72.0        0.0                1  5a8bc65990b245e5a138643cd4eb9837   
34525      72.0        0.0                1  5a8bc65990b245e5a138643cd4eb9837   
36096      96.0        0.0                1  3f207df678b143eea3cee63160fa8bed   
36568      72.0        0.0                1  5a8bc65990b245e5a138643cd4eb9837   
36772      72.0        0.0                1  5a8bc65990b245e5a138643cd4eb9837   

       offer_received     offer_type  offer_view  \
4086              1.0       discount         2.0   
4093              1.0       discount         2.0   
4138              1.0           bogo         2.0   
4198              1.0           bogo         2.0   
4230              1.0           bogo         2.0   
...               ...            ...         ...   
34489             1.0  informational         2.0   
34525             1.0  informational         2.0   
36096             1.0  informational         2.0   
36568             1.0  informational         2.0   
36772             1.0  informational         2.0   

                                 person  reward  time_completed  time_start  \
4086   d058f73bf8674a26a95227db098147b1     2.0           192.0       168.0   
4093   9c64321646bf40f6ad1bc98864eaa6b8     2.0           216.0       168.0   
4138   8c410d84af08408fb41f953c93ffac27     5.0           168.0       168.0   
4198   ef0dcb0bddc54f3595846a87bd1237ef    10.0           180.0       168.0   
4230   4324b549f593457696d336c2925ee12f     5.0           168.0       168.0   
...                                 ...     ...             ...         ...   
34489  28ddc2fd0f9344008828d2fd1d846e24     0.0           552.0       576.0   
34525  f07862abca4a4b8d98e0ef8ed8b808f5     0.0           504.0       504.0   
36096  8142c6f808944e7a87a18950c71b18ae     0.0           576.0       576.0   
36568  8a647583b38f4397a8870b70eaefcc08     0.0           594.0       576.0   
36772  3b3e4f3238db42eebb04b1f658b55a9f     0.0           594.0       576.0   

       time_view  total  time_validity  amount  
4086       180.0    7.0          336.0   27.23  
4093       174.0    7.0          336.0   20.16  
4138       204.0    7.0          288.0   11.93  
4198       600.0    7.0          336.0   32.03  
4230       204.0    7.0          288.0   25.86  
...          ...    ...            ...     ...  
34489      432.0    3.0          648.0    0.12  
34525      504.0    3.0          576.0   25.78  
36096      576.0    3.0          672.0    5.80

In [1605]:
normal_transactions.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/normal_transaction.csv') # trasactions without offer influence
offer_success.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/offer_success.csv', encoding='utf-8') #transactions and other data of successful offers
waste_offer_transaction.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/waste_offer_transaction.csv', encoding='utf-8')  #transactions data of transactions made without viewing offer
waste_offer.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/waste_offer.csv', encoding='utf-8') #total waste offers including waste_offer_transactions
profile_amount.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/profile_amount.csv', encoding='utf-8') #profile and total amount spend data
transaction_df.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/transaction_df.csv', encoding='utf-8') #original Transaction DF